# `mle-launcher`: Lightweight Cluster/Cloud VM Job Scheduling 🚂
### Author: [@RobertTLange](https://twitter.com/RobertTLange) [Last Update: October 2021][![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobertTLange/mle-launcher/blob/main/examples/getting_started.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

try:
    import mle_launcher
except:
    !pip install -q mle-launcher
    import mle_launcher

# Single Job Management with `MLEJob` 🚀

In [7]:
from mle_launcher import MLEJob

# Launch a python train_mnist.py -config base_config.json job
job = MLEJob(resource_to_run="slurm",
             job_filename="train.py",
             config_filename="base_config_1.json")

# Job Queue Management with `MLEQueue` 🚀🚀🚀

In [ ]:
from mle_launcher import MLEQueue

# Launch a queue of 4 jobs (2 configs x 2 seeds)
# python train_mnist.py -config base_config_1.json -seed 0
# python train_mnist.py -config base_config_1.json -seed 1
# python train_mnist.py -config base_config_2.json -seed 0
# python train_mnist.py -config base_config_2.json -seed 1
queue = MLEQueue(resource_to_run="slurm",
                 job_filename="train.py",
                 config_filenames=["base_config_1.json",
                                   "base_config_2.json"],
                 random_seeds=[0, 1])